# Conformal Inference
Julian Hsu
5jun2022

In this notebook we look at the idea of conformal inference from *Chernozhukov, Wuthrich, and Zhu* : https://arxiv.org/abs/1712.09089. 


In [3]:

import sys
import os
print (os.getcwd() )


import seaborn as sns

import pandas as pd
import numpy as np
import itertools
import math
import scipy
import functools
import sklearn as sk
from sklearn import impute, pipeline

%matplotlib inline
import matplotlib.pyplot as plt


pd.options.mode.chained_assignment = None  # default='warn'



/Users/hsujulia/Documents/GitHub/causalinference_crashcourse/Notebooks


In [4]:
## Function to automatically give us a panel dataset
def synth_panel(seed_number = 2123, 
                time_units = 10, 
                x_units = 100, 
                K_dim = 5,
               treatment_type = 'both',
               treatment_imp = 10,
               treatment_het = False):
    '''
    seed_number  random_number_set
    time_units   how many time periods?
    x_units      how many units?
    K_dim        how many covariates (assume uniform)
    treatment_type    how is treatment determined? {'X','Y', 'both'}
    treatment_imp     the level of treatment effect, constant for all post-treatment times
    '''
    ## Set the seed for random number
    xfun = np.random.RandomState(seed_number)
    
    ## Construct covariates, time-invariant fixed effects, and first period outcome
    X = xfun.uniform(low=0, high=1, size= (x_units, K_dim) )
    X_coef = xfun.uniform(low=-2, high=2, size = K_dim)
    x_list = ['x'+str(k) for k in range(K_dim)]
    
    
    fe = xfun.normal(0,1, size=x_units)
    
    output_df = pd.DataFrame(data=X, columns=x_list)
    output_df['fe'] = fe
    output_df['Y'] = np.dot(X,X_coef) + fe + xfun.normal(0,1, size=x_units)
    output_df['T'] = pd.to_datetime('2020/01/01')
    output_df['unitid'] =  np.arange(x_units).astype(int)

    decay = 0.60

    for t in range(1,time_units):    
        ## Create the panel by setting up an AR(1) type function.
        prev_date = (pd.to_datetime('2020/01/01')+pd.offsets.DateOffset(months=t-1))
        prev_data = decay*( output_df.loc[(output_df['T']==prev_date)][[r for r in output_df.columns if 'x' in r ]]     )
        ## perturb it a little with AR(1)
        prev_data += xfun.normal(0,1, size=(x_units,K_dim))
        prev_data['Y'] = decay*np.dot(prev_data,X_coef) + fe + xfun.normal(0,1, size=x_units)
        prev_data['T'] = pd.to_datetime('2020/01/01') + pd.offsets.DateOffset(months=t)
        prev_data['unitid'] = np.arange(x_units).astype(int)
        output_df = pd.concat([ output_df, prev_data])
    output_df['unitid'] = 'unit' + output_df['unitid'].apply(str)
    

    '''
    Decide the treatment effect based either the (i) covariates, (ii) pre-trend outcome, or (iii) both.
    
    Assign treatment to happen at the third to last time period
    '''
    if treatment_type=='X':
        input_treatment = x_list[:]
    elif treatment_type=='Y':
        input_treatment = 'Y'
    else:
        input_treatment = x_list[:] + ['Y']
        
    output_df['treatment_latent'] = 0
    treatment_date = (pd.to_datetime('2020/01/01')+pd.offsets.DateOffset(months=time_units-3))    
    
    latent_treatment = np.dot(output_df[input_treatment], 
                              xfun.uniform(-4,4, size=len(input_treatment))) +\
                             xfun.normal(0,1,size=len(output_df))
    output_df['treatment_latent'] = np.exp(latent_treatment) / (1+ np.exp(latent_treatment))
    
    ## First decide treatment in the treatment time, and then make it permanent.    
    treatment_latent_xsection = output_df.loc[(output_df['T'] == treatment_date)][['unitid','treatment_latent']]
    treatment_decision = (treatment_latent_xsection['treatment_latent'] >\
                          treatment_latent_xsection['treatment_latent'].quantile(q=0.75))
    treatment_ids = treatment_latent_xsection.loc[(treatment_decision==True)]['unitid']
    
    output_df['post'] = (output_df['T'] >= treatment_date).astype(float)
    output_df['treatment_units'] = (output_df['unitid'].isin(treatment_ids)).astype(float)
    
    output_df['treatment'] = 0
    output_df.loc[ (output_df['unitid'].isin(treatment_ids)) & (output_df['T'] >= treatment_date), 'treatment'  ] = 1

    '''
    Apply treatment effect. 
    if we allow treatment heterogeneity, assign treatment such that the ATE!=ATET
    '''
    if treatment_het==False:
        output_df['treatment_GT']  = treatment_imp
    else:
        ## Calculate the proportion treated
        treat_mean = output_df['treatment'].mean()
        output_df['treatment_GT'] = treatment_imp/2
        output_df.loc[(output_df['treatment']==1), 'treatment_GT'] = treatment_imp*2

        
    output_df.loc[( output_df['treatment']==1) , 'Y'] = output_df.loc[( output_df['treatment']==1)]['treatment_GT'] + output_df.loc[( output_df['treatment']==1)]['Y']

    return output_df

## Permutations to get P-Values
Implement the $iid$ permutation approach. Given a set of time period, go through different iterations in that time period. 

Our setup is for the number pre-treatment periods $T_0$ and the number of post-treatment periods $T_1$. $\hat{u}_t$ is the difference between the estimated proxy outcome and realized outcome.
$$S_q(\hat{u}) = (T_1^{-1/2}  \sum^T_{t = T_0 + 1}|\hat{u}_t|^q )^{1/q}$$
**Note that $S_q(\hat{u})$ is calculated without any permutations or scrambling of the data.** In order to get p-values, we need to do multiple permutations, each with their own instance of $S_q(\hat{u})$. Denote each of the permutations $\pi \in \Pi$ as $S_q(\hat{u}_{\pi})$. The p-value is then:

$$\hat{p} = 1 - \dfrac{1}{\Pi} \sum_{\pi \in \Pi} 1\{ S_q(\hat{u}_{\pi}) < S_q(\hat{u})  \} $$

We can also calculate the confidence interval by a grid search process. Consider a lot of different treatment effects - for example if you are interested in treatment effects that are constant in all post-treatment periods, it is just one number. Each treatment effect is a "null hypotheses" you incorporate when training the model, so you can calculate $\hat{p}$ by seeing whether it is different from that treatment effect "null hypothesis." Then estimate the confidence interval of $1-\alpha$ (or set, since we are doing a grid search) where those $\hat{p}$ are greater than tha False Positive Rate $\alpha$.


Therefore, our implementation algorithm is:
1. Don't scramble anything and train the model.
    1. Estimate the treatment effect.
    2. Estimate the residual and calculate $S_q({\hat{u})$.
    
2. Pick one way of scrambling the time periods, $\pi$;
3. Estimate the model based on that scrambling, $\pi$.  Calculate $S_q({\hat{u}_\pi)$.
4. Calculate the p-value, $\hat{p}$.
5. If we are interested, calculate the confidence set by 

For simplicity sake, let's just use synthetic control following Abadie, Diamond, and Hainmueller (2010), which does an unpenalized synthetic control (SC) model. We can later on incorporate difference-in-difference (DiD) models.

In [19]:
## Abadie/Diamond/Hainmueller    
from typing import List
from operator import add
from toolz import reduce, partial
from scipy.optimize import fmin_slsqp

## Define loss function
def loss_w(W, X, y) -> float:
    return np.sqrt(np.mean((y - X.dot(W))**2))

def get_w(X, y):
    ## Initialize at sample average with some noise
    w_start = [1/X.shape[1]]*X.shape[1]+np.random.uniform(-0.005,0.005, X.shape[1])
#     w_start = np.ones(X.shape[1])

    weights = fmin_slsqp(partial(loss_w, X=X, y=y),
                         np.array(w_start),
                         f_eqcons=lambda x: np.sum(x) - 1,
                         iter=50000, 
                         bounds=[(0.0, 1.0)]*len(w_start),
                         disp=False)
    return weights

def syncontrol(X1=None,X0=None,
               Y1=None,Y0=None,
              scaledown = None):
    '''
    X1   control post-treatment
    X0   control pre-treatment
    Y1   treatment post-treatment
    Y0   treatment pre-treatment
    scaledown   whether to divide everything by a number, defaulted to none
    '''
    
    if scaledown is None:
        adh_weights = get_w(X0, Y0)
    else:
        adh_weights = get_w(X0/scaledown, Y0/scaledown)
    ## Output the weights
    
    ## Output the counterfactual control outcome
    Y_hatpre = np.dot(X0,adh_weights)
    Y_hatpst = np.dot(X1,adh_weights)
    
    ## Output the average treatment effect on treated
    atet = np.average(Y1 - Y_hatpst)
    
    return {'weights': adh_weights, 'control_pre': Y_hatpre,
            'control_pst': Y_hatpst, 'atet': atet} 



In [20]:
def sc_data_from_synth(df):    
    df_pt = df[['unitid','T','Y']].pivot_table(columns='unitid', index='T', values='Y')
    df_pt.reset_index(inplace=True)
    treatment_time = df.loc[df['treatment']==1]['T'].min()
    treated_units = df.loc[(df['treatment_units']==1)]['unitid'].unique()
    control_pre = df_pt.loc[(df_pt['T']< treatment_time)][[e for e in df_pt.columns if e not in treated_units]]
    control_pst = df_pt.loc[(df_pt['T']>=treatment_time)][[e for e in df_pt.columns if e not in treated_units]]

    treat_pre = df_pt.loc[(df_pt['T']< treatment_time)][treated_units[0]]
    treat_pst = df_pt.loc[(df_pt['T']>=treatment_time)][treated_units[0]]
    return control_pre, control_pst, treat_pre, treat_pst


In [21]:
df = synth_panel()
control_pre, control_pst, treat_pre, treat_pst = sc_data_from_synth(df)

In [23]:
## Estimate the treatment effect without any perturbation
sc_output = syncontrol(X1=control_pst.drop(columns ='T'), X0=control_pre.drop(columns='T'),
                      Y1=treat_pst, Y0=treat_pre)
print(sc_output.keys())

## Calculate the residual
residual = np.concatenate( [sc_output['control_pre'] - treat_pre,
                 sc_output['control_pst'] - treat_pst])

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6327271025076378
            Iterations: 12
            Function evaluations: 916
            Gradient evaluations: 12
dict_keys(['weights', 'control_pre', 'control_pst', 'atet'])


In [ ]:
## Now we want to scramble the dates. Do this by replacing the date index.
scrambled_dates = np.random.choice( len(df['T'].unique()),len(df['T'].unique()), replace=False )

num_treated_time = len(df.loc[ df['treatment']==1]['T'].unique())


In [36]:
def scrambled_sc(df, scrambled_dates, num_treated_time):
    df_scrambled = df.copy()
    df_scrambled['T'].replace(df_scrambled['T'].unique(), scrambled_dates, inplace=True)
    df_scrambled['treatment'] = (df_scrambled['T']>=num_treated_time).astype(float)

    ## Create the SC data
    control_pre, control_pst, treat_pre, treat_pst = sc_data_from_synth(df_scrambled)

    ## Train the SC model
    sc_output = syncontrol(X1=control_pst.drop(columns ='T'), X0=control_pre.drop(columns='T'),
                          Y1=treat_pst, Y0=treat_pre)

    ## Calculate the residual
    residual = np.concatenate( [sc_output['control_pre'] - treat_pre,
                     sc_output['control_pst'] - treat_pst])
    return residual